In [135]:
import numpy as np
import cvxpy as cp

## Example from cvxpy 

In [136]:
n = 3
p = 3

np.random.seed(1)

C = np.random.randn(n,n)
A = []
b = []

for i in range(p):
    A.append(np.random.randn(n,n))
    b.append(np.random.randn())

In [137]:
X = cp.Variable((n,n), symmetric = True)

constraints = [X >> 0]
constraints += [
    cp.trace(A[i] @ X) == b[i] for i in range(p)
]

In [138]:
prob = cp.Problem(cp.Minimize(cp.trace(C @ X)), constraints)
prob.solve()

2.654348022626433

In [ ]:
X.value

array([[ 1.60805504, -0.59770125, -0.69575821],
       [-0.59770125,  0.22228555,  0.24689067],
       [-0.69575821,  0.24689067,  1.39679134]])

## Example with quantum state

In [ ]:
rho = np.array([[0.5, 0.2, 0.1], [0.2, 0.3, 0.], [0.1, 0., 0.2]])

X = cp.Variable((3,3), hermitian = True) #setting hermitian makes cvxpy think entries are complex

constraints = [X>>0, cp.trace(X) == 1]

objective = cp.real(cp.trace(rho @ X)) #so we need to take real part here so no complex in objective fct

prob = cp.Problem(cp.Maximize(objective), constraints)
prob.solve()

0.6402685566554495

In [ ]:
X.value

array([[0.71578654+0.j, 0.42071985+0.j, 0.16257992+0.j],
       [0.42071985+0.j, 0.24728669+0.j, 0.09556   +0.j],
       [0.16257992+0.j, 0.09556   +0.j, 0.03692677+0.j]])

In [ ]:
rho_0 = np.array([[0.7, 0.2], [0.2, 0.3]])  # Example state 1
rho_1 = np.array([[0.4, 0.1], [0.1, 0.6]])  # Example state 2

E0 = cp.Variable((2,2), hermitian=True)
E1 = cp.Variable((2,2), hermitian = True)

constraints = [E0 >> 0, E1 >> 0, E0 + E1 == np.eye(2)]

objective = cp.real(0.5 * cp.trace(rho_0 @ E0) + 0.5 * cp.trace(rho_1 @ E1))

prob = cp.Problem(cp.Maximize(objective), constraints)
prob.solve()



np.float64(0.6581148607722879)

In [ ]:
E0.value

array([[0.97434458+0.j, 0.15811486+0.j],
       [0.15811486+0.j, 0.02565544+0.j]])

In [ ]:
d = 2

phip = np.array([[1, 0, 0, 1], 
                     [0, 0, 0, 0], 
                     [0, 0, 0, 0], 
                     [1, 0, 0, 1]]) / d


J = cp.Variable((d**2, d**2), hermitian = True)

constraints = [J >> 0, cp.partial_trace(J, (d,d), 1) == np.eye(d)]

objective = cp.real(cp.trace(phip @ J))

prob = cp.Problem(cp.Maximize(objective), constraints)
prob.solve()

2.0000000195088443

In [ ]:
J.value

array([[ 1.00000001e+00+0.j, -1.42895070e-16+0.j,  3.39844948e-17+0.j,
         1.00000001e+00+0.j],
       [-1.42895070e-16+0.j, -6.49215334e-09+0.j, -4.50064862e-34+0.j,
        -3.41646567e-17+0.j],
       [ 3.39844948e-17+0.j, -4.50064862e-34+0.j, -6.49215029e-09+0.j,
        -7.47418209e-17+0.j],
       [ 1.00000001e+00+0.j, -3.41646567e-17+0.j, -7.47418209e-17+0.j,
         1.00000001e+00+0.j]])

In [ ]:
y0 = cp.Variable()
y1 = cp.Variable()
y2 = cp.Variable()
y3 = cp.Variable()
y4 = cp.Variable()

M2 = cp.bmat([[y0,y1,y2],
              [y1,y2,y3],
              [y2,y3,y4]])
M1 = cp.bmat([[y0,y1],[y1,y2]])

constraints = [M2 >> 0, M1 >> 0, y0 == 1]
constraints += [y1 >= -1, y1 <= 1]
constraints += [y3 >= -1, y3 <= 1]
constraints += [y4 >= 0, y4 <= 1]



objective = y4 - 2*y2 + y1

prob = cp.Problem(cp.Maximize(objective), constraints)
prob.solve()

np.float64(1.125000232787867)

In [ ]:
np.array([[2],[2]]).shape

(2, 1)

In [1]:
import simulation
import chaospy
import numpy as np
import cvxpy as cp

In [2]:
delta, p = simulation.getProbas(0.7, 1, 0, '1')

In [3]:
m_in = 10
m = int(m_in*2)
distribution = chaospy.Uniform(lower=0, upper=1)
t, w = chaospy.quadrature.radau(m_in,distribution,1.0)
t = t[0]

In [204]:
Gamma00 = cp.Variable((28,28), complex = False)
Gamma01 = cp.Variable((28,28), complex = False)
Gamma10 = cp.Variable((28,28), complex = False)
Gamma11 = cp.Variable((28,28), complex = False)

Gamma = cp.kron(Gamma00, np.array([[1,0],[0,0]])) + cp.kron(Gamma01, np.array([[0,1],[0,0]])) + cp.kron(Gamma10, np.array([[0,0],[1,0]])) + cp.kron(Gamma11, np.array([[0,0],[0,1]]))

constraints = [Gamma >> 0]

In [205]:
def getA(px, ti, taui):
    A = np.zeros((28,28))
    
    indices = [(1,4), (1,5), (2,6), (2,7), (3,8), (3,9), (1,16), (2,18), (3,20), (0,17), (0,19), (0,21)]

    for id in indices:
        if id in [(1,4), (1,5), (2,6), (2,7), (3,8), (3,9)]:
            A[id[0], id[1]] = (1-px) * taui
            # A[id[1], id[0]] = (1-px) * taui
        elif id in [(1,16), (2,18), (3,20)]:
            A[id[0], id[1]] = (1-px) * taui * (1-ti)
            # A[id[1], id[0]] = (1-px) * taui * (1-ti)
        else:
            A[id[0], id[1]] = (1-px) * taui * (ti)

    indices = [(1,10), (1,11), (2,12), (2,13), (3,14), (3,15), (1,22), (2,24), (3,26), (0,23), (0,25), (0,27)]

    B = np.zeros((28,28))
    for id in indices: 
        if id in [(1,10), (1,11), (2,12), (2,13), (3,14), (3,15)]:
            B[id[0], id[1]] = (px) * taui
            # A[id[1] + 28, id[0] + 28] = (px) * taui
        elif id in [(1,22), (2,24), (3,26)]:
            B[id[0], id[1]] = (px) * taui * (1-ti)
            # A[id[1] + 28, id[0] + 28] = (px) * taui * (1-ti)
        else:
            B[id[0], id[1]] = (px) * taui * (ti)
    
    return np.kron(A, np.array([[1,0],[0,0]])).T + np.kron(B, np.array([[0,0], [0,1]])).T

In [206]:
def getCs(Gamma00, Gamma11, p, constraints):
    Cs = {b:{x :[np.zeros((28,28)) for _ in range(3)] for x in range(2)} for b in range(3)}
    ids = {0: [(0,1), (1,0), (1,1)], 1: [(0,2), (2,0), (2,2)], 2: [(0,3), (3,0), (3,3)]}
    
    for b in range(3):
        for x in range(2):
            Cs[b][x][0][ids[b][0]] = 1
            Cs[b][x][1][ids[b][1]] = 1
            Cs[b][x][2][ids[b][2]] = 1
            if x == 0:
                for i in range(3):
                    # C = np.kron(Cs[b][x][i], np.array([[1,0], [0,0]])).T
                    constraints += [(cp.trace(Gamma00 @ Cs[b][x][i].T)) == p[b][x]]
            elif x == 1:
                for i in range(3):
                    # C = np.kron(Cs[b][x][i], np.array([[0,0], [0,1]])).T
                    constraints += [(cp.trace(Gamma11 @ Cs[b][x][i].T)) == p[b][x]]

In [207]:
getCs(Gamma00, Gamma11, p, constraints)

In [208]:
Ds = [np.zeros((28,28)) for _ in range(4)]

Ds[0][0,0] = 1
Ds[1][0,0] = 1
Ds[2][0,0] = 1
Ds[3][0,0] = 1

constraints += [(cp.trace(Gamma00 @ Ds[0].T)) == 1]
constraints += [(cp.trace(Gamma01 @ Ds[1].T)) == delta]
constraints += [(cp.trace(Gamma10 @ Ds[2].T)) == delta]
constraints += [(cp.trace(Gamma11 @ Ds[3].T)) == 1]

In [209]:
import itertools
def getCombs(indices):
# All 27 possible combinations of one from each group
    raw_combinations = itertools.product(indices[0], indices[1], indices[2])

    # Use sets to remove permutations that are just re-ordered versions of each other
    unique_sets = set()

    for combo in raw_combinations:
        # Sort to make permutations identical
        sorted_combo = tuple(sorted(combo))
        unique_sets.add(sorted_combo)

    # Convert to list if needed
    unique_combinations = list(unique_sets)

    return unique_combinations

In [210]:
Ms = [[np.zeros((28,28)) for _ in range(27)] for _ in range(4)]

indices = [[(0,1), (1,0), (1,1)], [(0,2), (2,0), (2,2)], [(0,3), (3,0), (3,3)]]
combis = getCombs(indices)

for i in range(4):
    for j, comb in enumerate(combis):
        Ms[i][j][comb[0]] = 1
        Ms[i][j][comb[1]] = 1
        Ms[i][j][comb[2]] = 1

basis = [np.array([[1,0], [0,0]]), np.array([[0,1], [0,0]]), np.array([[0,0], [1,0]]), np.array([[0,0], [0,1]])]
G = [1, delta, delta, 1]

constraints += [cp.trace(Gamma00 @ Ms[0][j].T) == 1 for j in range(27)]
constraints += [cp.trace(Gamma01 @ Ms[1][j].T) == delta for j in range(27)]
constraints += [cp.trace(Gamma10 @ Ms[2][j].T) == delta for j in range(27)]
constraints += [cp.trace(Gamma11 @ Ms[3][j].T) == 1 for j in range(27)]


In [ ]:
import re

# Your operator lists
S = [
    "id", "M0", "M1", "M2", 
    "Z00", "Z00dag", "Z10", "Z10dag", "Z20", "Z20dag", 
    "Z01", "Z01dag", "Z11", "Z11dag", "Z21", "Z21dag", 
    "Z00dagZ00", "Z00Z00dag", "Z10dagZ10", "Z10Z10dag", 
    "Z20dagZ20", "Z20Z20dag", "Z01dagZ01", "Z01Z01dag", 
    "Z11dagZ11", "Z11Z11dag", "Z21dagZ21", "Z21Z21dag"
]

S_ = [
    "id", "M0", "M1", "M2", 
    "Z00dag", "Z00", "Z10dag", "Z10", "Z20dag", "Z20", 
    "Z01dag", "Z01", "Z11dag", "Z11", "Z21dag", "Z21", 
    "Z00dagZ00", "Z00Z00dag", "Z10dagZ10", "Z10Z10dag", 
    "Z20dagZ20", "Z20Z20dag", "Z01dagZ01", "Z01Z01dag", 
    "Z11dagZ11", "Z11Z11dag", "Z21dagZ21", "Z21Z21dag"
]

pattern = re.compile(r"M\d|Z\d\d(?:dag)?|id")

def split_factors(op):
    return pattern.findall(op)

def move_Ms_to_right(factors):
    factors = factors[:]
    changed = True
    while changed:
        changed = False
        for i in range(len(factors)-1):
            if factors[i].startswith('M') and (factors[i+1].startswith('Z')):
                factors[i], factors[i+1] = factors[i+1], factors[i]
                changed = True
    return factors

def simplify_Ms(M_factors):
    if not M_factors:
        return 'id'
    unique = set(M_factors)
    if len(unique) == 1:
        return M_factors[0]
    return None  # zero

def canonical_product(op1, op2):
    factors1 = split_factors(op1)
    factors2 = split_factors(op2)
    all_factors = factors1 + factors2
    
    # Remove all 'id' factors (neutral element)
    all_factors = [f for f in all_factors if f != 'id']
    
    # Move all Ms to the right
    all_factors = move_Ms_to_right(all_factors)
    
    # Separate Ms and Z factors
    M_factors = [f for f in all_factors if f.startswith('M')]
    Z_factors = [f for f in all_factors if not f.startswith('M')]
    
    # Simplify Ms
    M_simpl = simplify_Ms(M_factors)
    if M_simpl is None:
        return None
    
    return (tuple(Z_factors), M_simpl)

product_map = {}
zero_list = []
for i, op1 in enumerate(S_):
    for j, op2 in enumerate(S):
        cform = canonical_product(op1, op2)
        if cform is None:
            zero_list.append((i,j))
        else:
            product_map[(i,j)] = cform

equal_list = []
checked_pairs = set()
keys = list(product_map.keys())

for idx1 in range(len(keys)):
    for idx2 in range(idx1+1, len(keys)):
        key1 = keys[idx1]
        key2 = keys[idx2]
        if product_map[key1] == product_map[key2]:
            if (key1, key2) not in checked_pairs and (key2, key1) not in checked_pairs:
                equal_list.append((key1, key2))
                checked_pairs.add((key1, key2))

print("Zero products (index pairs):")
print(sorted(zero_list))

print("\nEqual products (distinct pairs of index pairs):")
print(sorted(equal_list))

Zero products (index pairs):
[(1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 2)]

Equal products (distinct pairs of index pairs):
[((0, 1), (1, 0)), ((0, 1), (1, 1)), ((0, 2), (2, 0)), ((0, 2), (2, 2)), ((0, 3), (3, 0)), ((0, 3), (3, 3)), ((0, 4), (5, 0)), ((0, 5), (4, 0)), ((0, 6), (7, 0)), ((0, 7), (6, 0)), ((0, 8), (9, 0)), ((0, 9), (8, 0)), ((0, 10), (11, 0)), ((0, 11), (10, 0)), ((0, 12), (13, 0)), ((0, 13), (12, 0)), ((0, 14), (15, 0)), ((0, 15), (14, 0)), ((0, 16), (4, 4)), ((0, 16), (16, 0)), ((0, 17), (5, 5)), ((0, 17), (17, 0)), ((0, 18), (6, 6)), ((0, 18), (18, 0)), ((0, 19), (7, 7)), ((0, 19), (19, 0)), ((0, 20), (8, 8)), ((0, 20), (20, 0)), ((0, 21), (9, 9)), ((0, 21), (21, 0)), ((0, 22), (10, 10)), ((0, 22), (22, 0)), ((0, 23), (11, 11)), ((0, 23), (23, 0)), ((0, 24), (12, 12)), ((0, 24), (24, 0)), ((0, 25), (13, 13)), ((0, 25), (25, 0)), ((0, 26), (14, 14)), ((0, 26), (26, 0)), ((0, 27), (15, 15)), ((0, 27), (27, 0)), ((1, 0), (1, 1)), ((1, 4), (5, 1)), ((1, 5), (4, 1)), ((

In [212]:
basis = [np.array([[1,0], [0,0]]), np.array([[0,1], [0,0]]), np.array([[0,0], [1,0]]), np.array([[0,0], [0,1]])]
Gammas = [Gamma00, Gamma01, Gamma10, Gamma11]

for id in zero_list:
    for i in range(4):
        F = np.zeros((28,28))
        F[id] = 1
        constraints += [(cp.trace(Gammas[i] @ F.T)) == 0]

equality_constraints = []

for id in equal_list:
    if id[0] not in [(0,1), (1,0), (1,1), (0,2), (2,0), (2,2), (0,3), (3,0), (3,3)] and id[1] not in [(0,1), (1,0), (1,1), (0,2), (2,0), (2,2), (0,3), (3,0), (3,3)]:
        for i in range(4):
            F = np.zeros((28,28))
            F[id[0]] = 1
            F[id[1]] = -1 
            constraints += [(cp.trace(Gammas[i] @ F.T)) == 0]

In [213]:
t0 = t[0]
tau0 = (w[0]/(t[0]*np.log(2)))
A = getA(7/8, t0, tau0)

obj = (cp.trace(Gamma @ A))

In [214]:
eigvals = np.linalg.eigvalsh(A)
print(min(eigvals), max(eigvals))

-5.597683374851717 5.597683374851717


In [215]:
constraints += equality_constraints

In [216]:
constraints += [(cp.trace(Gamma)) <= 4.0000003]
prob = cp.Problem(cp.Minimize(obj), constraints)
mosek_params = {
    "MSK_DPAR_INTPNT_TOL_REL_GAP": 1e-4,
    "MSK_DPAR_INTPNT_TOL_PFEAS": 1e-4,
    "MSK_DPAR_INTPNT_TOL_DFEAS": 1e-4,
    "MSK_DPAR_INTPNT_TOL_MU_RED": 1e-4,
}
prob.solve(solver='MOSEK', verbose=True, mosek_params = mosek_params)

                                     CVXPY                                     
                                     v1.6.4                                    
(CVXPY) May 28 05:54:56 PM: Your problem has 3136 variables, 3819 constraints, and 0 parameters.


(CVXPY) May 28 05:54:56 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 28 05:54:56 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 28 05:54:56 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) May 28 05:54:56 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 28 05:54:56 PM: Compiling problem (target solver=MOSEK).
(CVXPY) May 28 05:54:56 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> MOSEK
(CVXPY) May 28 05:54:56 PM: Applying reduction Dcp2Cone
(CVXPY) May 28 05:54:56 PM: Applying reduction CvxAttr2Constr
(CVXPY) May 28 05:54:56 PM: Applyi

SolverError: Solver 'MOSEK' failed. Try another solver, or solve with verbose=True for more information.

In [72]:
Ga = Gamma.value
(np.diag(Ga))

array([ 9.99999983e-01,  9.99999980e-01, -2.77517893e-09,  2.49143933e-10,
        8.26577366e-09,  3.87373598e-01,  9.99999976e-01,  6.12626382e-01,
        1.59136780e-05,  1.81200909e-05,  1.58728664e-05,  1.81469772e-05,
        1.59662972e-05,  1.80711304e-05,  1.59408602e-05,  1.80994772e-05,
        2.50529053e-02,  1.53536586e-02,  2.50584672e-02,  1.53547237e-02,
        1.60008216e-05,  1.87762580e-05,  1.60188567e-05,  1.87144460e-05,
        1.02235126e-04,  1.27624992e-01,  2.14916612e-05,  1.27662624e-01,
        1.54317370e-01,  9.52161962e-02,  1.54316244e-01,  9.52050160e-02,
        1.88647398e-05,  1.75444403e-05,  1.87703547e-05,  1.75454975e-05,
        1.89655492e-05,  1.75634369e-05,  1.88246119e-05,  1.75517138e-05,
        6.47384973e-04,  4.01818554e-04,  6.47066832e-04,  4.01819825e-04,
        1.86805772e-05,  1.93667201e-05,  1.86120069e-05,  1.84115640e-05,
        5.39836465e-05,  4.27184106e-02,  2.23214575e-05,  4.20956286e-02,
        2.38376085e-02,  

In [ ]:
p, delta

({0: {0: 0.0, 1: 0.0},
  1: {0: 0, 1: 0.3873736058155839},
  2: {0: 1, 1: 0.6126263941844161}},
 0.7827045382418681)

In [ ]:
traces = np.linspace(1.21, 1.7, 50)
for tr in traces:
    constraints += [(cp.trace(Gamma)) <= tr]
    prob = cp.Problem(cp.Minimize(obj), constraints)
    prob.solve(solver='MOSEK', verbose=False)
    if prob.status == 'infeasible':
        print(f'inf - {tr}')
        pass
    else:
        print(prob.value, tr)
        break
    constraints.pop()

SolverError: Solver 'MOSEK' failed. Try another solver, or solve with verbose=True for more information.

In [ ]:
constraints_base = constraints  # your base constraints without equality constraints
  # list of all equality constraints

# Test feasibility with base only
prob = cp.Problem(cp.Minimize(0), constraints_base)
status = prob.solve(solver='MOSEK', verbose=False)
print("Base constraints status:", status)

# Add equality constraints one by one or in groups
for i, eq_constr in enumerate(equality_constraints):
    constraints_test = constraints_base + equality_constraints[:i+1]
    prob = cp.Problem(cp.Minimize(0), constraints_test)
    status = prob.solve(solver='MOSEK', verbose=False)
    print(f"After adding equality constraint {i+1}, status:", status)
    if status == 'infeasible':
        print("Infeasibility found at constraint index", i)
        break


Base constraints status: inf


In [ ]:
equal_list[12]

((0, 10), (11, 0))

In [ ]:
a = np.zeros((28,28))

for id in zero_list:
    a[id] = 1
for id in equal_list:
    a[id[0]] = 1
    a[id[1]] = 1

for i in range(28):
    for j in range(28):
        if a[i,j] == 0:
            print(i,j)

0 0
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
4 14
4 15
4 20
4 21
4 22
4 23
4 24
4 25
4 26
4 27
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
5 14
5 15
5 20
5 21
5 22
5 23
5 24
5 25
5 26
5 27
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
6 12
6 13
6 14
6 15
6 16
6 17
6 18
6 19
6 24
6 25
6 26
6 27
7 4
7 5
7 6
7 7
7 8
7 9
7 10
7 11
7 12
7 13
7 14
7 15
7 16
7 17
7 18
7 19
7 24
7 25
7 26
7 27
8 4
8 5
8 6
8 7
8 8
8 9
8 10
8 11
8 12
8 13
8 14
8 15
8 16
8 17
8 18
8 19
8 20
8 21
8 22
8 23
9 4
9 5
9 6
9 7
9 8
9 9
9 10
9 11
9 12
9 13
9 14
9 15
9 16
9 17
9 18
9 19
9 20
9 21
9 22
9 23
10 4
10 5
10 6
10 7
10 8
10 9
10 10
10 11
10 12
10 13
10 14
10 15
10 20
10 21
10 22
10 23
10 24
10 25
10 26
10 27
11 4
11 5
11 6
11 7
11 8
11 9
11 10
11 11
11 12
11 13
11 14
11 15
11 20
11 21
11 22
11 23
11 24
11 25
11 26
11 27
12 4
12 5
12 6
12 7
12 8
12 9
12 10
12 11
12 12
12 13
12 14
12 15
12 16
12 17
12 18
12 19
12 24
12 25
12 26
12 27
13 4
13 5
13 6
13 7
13 8
13 9
13 10
13 11
13 12
13 13
13 14
13 15
13 16
13 17
13 18
13 19
